In [ ]:
# === Chunk 1: Setup & Daten laden ===
# (Dieser Chunk lädt die ROHE Feature Importance)

import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings  # KORREKTUR: Fehlender Import
import pingouin as pg
warnings.filterwarnings('ignore') # Jetzt funktioniert dieser Aufruf

# --- 1. Pfad-Setup (Standard-Snippet) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur: break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

try:
    # WICHTIG: _full_df laden! (benötigt angepasstes io_timesplits.py)
    from src.io_timesplits import load_fi_full_df
except ImportError:
    print("FEHLER: src-Module nicht gefunden. Stellen Sie sicher, dass 'src' im PROJECT_ROOT liegt.")
    # Fallback für den Fall, dass der Pfad manuell gesetzt werden muss
    PROJECT_ROOT = Path.cwd().parent  # Passen Sie dies bei Bedarf an
    sys.path.append(str(PROJECT_ROOT))
    from src.io_timesplits import load_fi_full_df

# --- 2. Daten laden ---
try:
    # fi_full_df.parquet enthält die rohen, monatlichen Wichtigkeiten
    df_fi_raw = load_fi_full_df()
    print(f"Rohe FI-Daten geladen: {df_fi_raw.shape}")
    display(df_fi_raw.head())
except FileNotFoundError:
    print("FEHLER: 'outputs/feature_importance/fi_full_df.parquet' nicht gefunden.")
    print("Bitte führen Sie zuerst 'feature_importance.ipynb' aus.")

# Wir fokussieren uns auf die Konsens-Wichtigkeit
df_fi = df_fi_raw[["t", "feature", "importance_consensus"]].copy()

In [ ]:
# === Chunk 2: Top-K Features identifizieren ===
# (Basierend auf der DURCHSCHNITTLICHEN Wichtigkeit)

K = 20  # Anzahl der Top-Features, die wir tracken wollen

# Berechne durchschnittliche Wichtigkeit über alle Zeiten
mean_importance = df_fi.groupby("feature")["importance_consensus"].mean()
top_k_features = mean_importance.nlargest(K).index.tolist()

print(f"Top {K} Features (basierend auf Ø-Wichtigkeit):")
print(top_k_features)

In [ ]:
# === Chunk 3: Rang-Berechnung & Stabilitäts-Plot ("Spaghetti-Plot") (KORRIGIERT) ===

# KORREKTUR: Rang über den *gesamten* monatlichen Datensatz berechnen,
# DANN filtern. Das gibt den "wahren" Rang und behebt die
# Inkonsistenz zwischen Code und Kommentar.

# 1. Berechne den Rang über ALLE Features für jeden Monat
# (ascending=False -> hohe Wichtigkeit = niedriger Rang)
df_fi["rank"] = df_fi.groupby("t")["importance_consensus"].rank(ascending=False, method="min")

# 2. Filtere den DataFrame auf die K Features, die uns interessieren
df_top_k_ranks = df_fi[df_fi["feature"].isin(top_k_features)]

# 3. Pivot für Plot
df_pivot = df_top_k_ranks.pivot(index="t", columns="feature", values="rank")

# 4. Plot
plt.figure(figsize=(16, 10))
sns.lineplot(data=df_pivot, dashes=False, legend="full")
plt.title(f"Rolling Rank Stability (Top {K} Features)", fontsize=16)
plt.ylabel("Wichtigkeits-Rang (Niedriger ist besser)")
plt.xlabel("Zeit")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.gca().invert_yaxis() # Niedrige Ränge (z.B. 1) oben
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# === Chunk 4: Deskriptive Stabilitätsmetrik (Rang-StdAbw) ===

# Berechne die Standardabweichung des Rangs für jedes Feature
# (Verwendet das gepivotete df_pivot aus dem vorherigen Chunk)
rank_stability = df_pivot.std().sort_values(ascending=True)

print(f"Stabilität der Top-{K} Features (StdAbw des Rangs):")
print("(Niedriger Wert = Stabiler)")
display(rank_stability.to_frame("Rank_StdDev"))

In [ ]:
# === Chunk 5: Formaler Konkordanz-Test (Kendall's W) ===

# 1. Pivot auf Wichtigkeit (nicht Rang) für die Top-K Features
df_pivot_imp = df_top_k_ranks.pivot(index="t", columns="feature", values="importance_consensus").fillna(0)

# 2. Resample in Blöcke (z.B. 2-Jahres-Blöcke)
# '2A' = 2 Jahre (endet am Jahresende), 'mean()' = durchschnittliche Wichtigkeit in dem Block
df_blocks = df_pivot_imp.resample("2A").mean().dropna()

# 3. Berechne Kendall's W
# `pg.kendall_w` erwartet Daten im Format: Subjekte (Features) x Rater (Zeitblöcke)
# Wir müssen transponieren
df_blocks_t = df_blocks.transpose()

w_test = pg.kendall_w(data=df_blocks_t)

print(f"Kendall's W (Konkordanz der Feature-Wichtigkeit über {len(df_blocks)} Zeitblöcke)")
print(f"W: {w_test['W'].item():.4f}")
print(f"p-Wert: {w_test['pval'].item():.4f}")

# Interpretation:
# W nahe 1: Perfekte Übereinstimmung (Features sind immer gleich wichtig)
# W nahe 0: Keine Übereinstimmung (Chaos)